## Blowup of Maps

We define the class `SelfMap` which allows to create a germ of endomorphism of K^n, for some field K; we can change variables to center them around a different point or by conjugating with a linear map.

The methods `blowup` and `blowup_ax` allow to blow-up a point or an affine subspace whose direction is given by some of the variables equal to 0.

We can compute the series expansion around a point and print it in a user-friendly way, in case the map is tangent to the identity.

In [1]:
class SelfMap:
    def __init__(self,comps,varb,params=None, inplace=False, base_field=QQ, ttti=True, degree_offset=2):
        self.nvars=len(varb)
        self.variables=varb
        self.comps=comps
        self.inplace=inplace
        self.params=params
        self.base_field=base_field
        if self.params is None:
            self.AusRing=self.base_field
            self.AusField=self.base_field
        else:
            self.AusRing=PolynomialRing(self.base_field,self.params)
            self.AusField=FractionField(self.AusRing)
            
        self.PolRing=PolynomialRing(self.AusField,self.variables)
        self.ttti=ttti
        self.degree_offset=degree_offset
        
    def get_first_hom(self, center=None):
        if center is None:
            center=[0]*self.nvars
        H=self.translate(center, inplace=False)    
        if self.ttti:
            i=2
            start=[(g-v).taylor(*zip(self.variables,[0]*self.nvars),i) for g,v in zip(H, self.variables)]
            while not set([i]).issubset(set([self.PolRing(s).degree() for s in start])):
                i+=1
                start=[(g-v).taylor(*zip(self.variables,[0]*self.nvars),i) for g,v in zip(H, self.variables)]
            return i,start
        else:
            i=0
            start=[g.taylor(*zip(self.variables,[0]*self.nvars),i) for g in H]
            while start==[0]*self.nvars:
                i+=1
                start=[g.taylor(*zip(self.variables,[0]*self.nvars),i) for g in H]
            return i,start
    
    def translate(self,center, inplace=None,out_map=False):
        if inplace is None:
            inplace=self.inplace
        if center==[0]*self.nvars:
            return self.comps
        coord_change=dict([(v,v+c) for (v,c) in zip(self.variables,center)])
        if (inplace and (not out_map)):
            self.comps=[f.subs(coord_change)-c for (f,c) in zip(self.comps, center)]
            
            return self.comps
        elif (inplace and out_map):
            self.comps=[f.subs(coord_change)-c for (f,c) in zip(self.comps, center)]
        
            return SelfMap(self.comps,self.variables, params=self.params,inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
        elif ((not inplace) and (not out_map)):
            return [f.subs(coord_change)-c for (f,c) in zip(self.comps, center)]
        else:
            return SelfMap([f.subs(coord_change)-c for (f,c) in zip(self.comps, center)],self.variables,params=self.params,
                       inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
        
    
    
    def blowup(self,center,chart,inplace=None,out_map=False):
        if inplace is None:
            inplace=self.inplace
        shifted=self.translate(center, inplace=False)
        coord_change=dict([(v,v if v==self.variables[chart-1] else v*self.variables[chart-1]) for v in self.variables])
        denom=shifted[chart-1].subs(coord_change)
        if (inplace and (not out_map)):
            self.comps=[denom if j==chart else g.subs(coord_change)/denom 
                        for (g,j) in zip(shifted,list(range(1,self.nvars+1)))]
            
            return self.comps
        elif (inplace and out_map):
            self.comps=[denom if j==chart else g.subs(coord_change)/denom 
                        for (g,j) in zip(shifted,list(range(1,self.nvars+1)))]
            
            return SelfMap(self.comps, self.variables,params=self.params,inplace=self.inplace,
                           base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
        elif ((not inplace) and (not out_map)):
            return [denom if j==chart else g.subs(coord_change)/denom 
                    for (g,j) in zip(shifted,list(range(1,self.nvars+1)))]
        else:
            return SelfMap([denom if j==chart else g.subs(coord_change)/denom 
                        for (g,j) in zip(shifted,list(range(1,self.nvars+1)))], self.variables,params=self.params,
                       inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
    
    
    def blowup_ax(self,center, ax,chart,inplace=None,out_map=False):
        if inplace is None:
            inplace=self.inplace
        shifted=self.translate(center, inplace=False)
        coord_change=dict([(v,v if (j+1 in ax+[chart]) else v*self.variables[chart-1]) 
                           for (v,j) in zip(self.variables, list(range(self.nvars)))])
        denom=shifted[chart-1].subs(coord_change)
        if (inplace and (not out_map)):
            self.comps=[g.subs(coord_change) if (j+1 in ax+[chart]) else g.subs(coord_change)/denom for (g,j) in
                        zip(shifted,list(range(self.nvars)))]
            
            return self.comps
        elif (inplace and out_map):
            self.comps=[g.subs(coord_change) if (j+1 in ax+[chart]) else g.subs(coord_change)/denom for (g,j) in 
                        zip(shifted,list(range(self.nvars)))]
            
            return SelfMap(self.comps,self.variables, params=self.params,inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
        elif ((not inplace) and (not out_map)):
            return [g.subs(coord_change) if (j+1 in ax+[chart]) else g.subs(coord_change)/denom for (g,j) in 
                    zip(shifted,list(range(self.nvars)))]
        else:
            return SelfMap([g.subs(coord_change) if (j+1 in ax+[chart]) else g.subs(coord_change)/denom for (g,j) in 
                            zip(shifted,list(range(self.nvars)))], self.variables,params=self.params,
                           inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,
                           degree_offset=self.degree_offset)
    
    
    def lin_change(self, center, change, inplace=None,out_map=False):
        if inplace is None:
            inplace=self.inplace
        shifted=self.translate(center, inplace=False)
        conj_change=Matrix(change).inverse()
        coord_change=dict(zip(self.variables, list(Matrix(change)*vector(self.variables))))
        if (inplace and (not out_map)):
            self.comps=list(conj_change*vector([c.subs(coord_change) for c in self.comps]))
            
            return self.comps
        elif (inplace and out_map):
            self.comps=list(conj_change*vector([c.subs(coord_change) for c in self.comps]))
            
            return SelfMap(self.comps,self.variables, params=self.params,inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
        elif ((not inplace) and (not out_map)):
            return list(conj_change*vector([c.subs(coord_change) for c in self.comps]))
        else:
            return SelfMap(list(conj_change*vector([c.subs(coord_change) for c in self.comps])),self.variables,params=self.params,inplace=self.inplace,base_field=self.base_field,ttti=self.ttti,degree_offset=self.degree_offset)
     
    def expansion(self, center, order):
        H=self.translate(center, inplace=False)
        return [c.taylor(*zip(self.variables,[0]*self.nvars), order) for c in H]
    
    def exp_by_deg(self, center, order):
        exp=self.expansion(center, order)
        H=[]
        for f in exp:
            g=self.PolRing(f)
            d = [0 for j in range(order+1)]
            for coeff,monom in g:
                if monom.degree()<order+1:
                    d[monom.degree()] += coeff * monom
                else:
                    break
            H.append(d)
        return H
    
    def ttti_exp(self,center, order,sep_fact=False):
        exp=self.expansion(center, order)
        H=[]
        cftot=[]
        cf=0
        for f in exp:
            if sep_fact:
                cf=0
            g=self.PolRing(f)
            d = [0 for j in range(order+1)]
            for coeff,monom in g:
                if monom.degree()<order+1:
                    d[monom.degree()] += coeff * monom
                    if monom.degree()>1:
                        cf=cf.gcd(monom)
                else:
                    break
            H.append(d)
            cftot.append(cf)
        if sep_fact:
            return cftot, H
        else:
            return [cftot[2]]*3,H
        
    def print_ttti_exp(self,center, order, out_exp=False, sep_fact=False, rem_fact=False):
        cftot, H=self.ttti_exp(center, order,sep_fact)
        print('*** Expansion around the point '+str(center)+' up to order '+str(order)+'\n')
        for (cf,h) in zip(cftot,H):
            component=str(h[1])+' + '+str(cf)+' * ('
            offset=len(component)
            for e in h[2:]:
                if not e==0:
                    add=str((self.PolRing(e/cf)))
                    if (component[-1]=='(' and add[0]=='-'):
                        component+=' - '+add[1:]
                    elif component[-1]=='(':
                        component+='   '+add
                    elif (add[0]=='+' or add[0]=='-'):
                        component+=' '+add[0]+' '+add[1:]
                    else:
                        component+=' + '+add
                    component+='\n'+(' '*offset)
            component+=' + ... )'
            print(component)
        if rem_fact:
            print('\n\n*** Common factor of terms from degree '+str([c.degree()+3 for c in cftot])+' by component')
            print('*** Excluding the common factors '+str(cftot)+'\n')
            for cf,h in zip(cftot,H):
                cfr=0
                for e in h[cf.degree()+3:]:
                    cfr=cfr.gcd(e)
                print('In the '+str(h[1])+' component: '+str(self.PolRing(cfr/cf)))
        if out_exp:
            return cftot,H
        else:
            return
    
    def get_sat_first_hom(self,center,div=0,prec=1):
        order, first_hom=self.get_first_hom(center)
        cftot,H=self.ttti_exp(center, order+self.degree_offset)
        cf=cftot[0].gcd(self.PolRing(div^(order)))
        return [self.PolRing(h/cf) for h in [sum(hh[order:order+prec]) for hh in H]]
    
    def get_char_dirs(self, center):
        sat_first_hom=self.get_sat_first_hom(center)
        nu=max([s.degree() for s in sat_first_hom])
        proj_space=ProjectiveSpace(self.nvars,self.base_field,','.join([str(v) for v in self.variables+[var('lambda_')]]))
        char_dirs_comps=proj_space.subscheme([h-lambda_^(nu-1)*v for h,v in zip(sat_first_hom,self.variables)]).irreducible_components()
        char_dirs=[]
        dicritic_comps=[]
        for cd_sch in char_dirs_comps:
            if cd_sch.dimension()==0:
                cd=list(cd_sch.rational_points()[0])
                if cd[:-1]!=[0]*self.nvars:
                    char_dirs.append((cd[:-1],cd[-1]))
            else:
                dicritic_comps.append((cd_sch.dimension(),cd_sch.defining_polynomials()))
        return char_dirs,dicritic_comps
        

We work out an example, depending on some parameters; first of all, we compute the characteristic directions.

In [2]:
var('x,y,z,R_040,Q_004,P_004,P_040,x0,y0,z0,R_004')

example=SelfMap([x+y*z*(y-z)+P_004*z^4+P_040*y^4,y+(x^2-z^2)*x+Q_004*z^4, z+x*z*(y-z)+R_040*y^4+R_004*z^4],[x,y,z], 
                inplace=False, params=[P_004,P_040,Q_004,R_040,R_004,x0,y0,z0])
example.get_char_dirs([0,0,0])

([([0, 1, 0], 0),
  ([0, 0, 1], 0),
  ([0, 1, 1], 0),
  ([1, 1, 1], 0),
  ([-1, 1, 1], 0)],
 [])

### First blow-up (of the origin)

In [3]:
print('***** Characteristic direction [0,0,1]\n')
X1_z=example.blowup([0,0,0],3,out_map=True)
X1_y=example.blowup([0,0,0],2,out_map=True)
X1_z.print_ttti_exp([0,0,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X1_z.get_sat_first_hom([0,0,0]))

***** Characteristic direction [0,0,1]

*** Expansion around the point [0, 0, 0] up to order 10

x + z^2 * ( - y + P_004*z
            + x^2 + y^2 + (-R_004)*x*z
            - x^2*y
            - x*y*z^2 + P_004*x*z^3 + R_004*y*z^3 + (-P_004*R_004)*z^4
            + P_040*y^4*z + x^3*z^2 + 2*x*y^2*z^2 + (-2*R_004)*x^2*z^3 + (-P_004)*x*y*z^3 + (-R_004)*y^2*z^3 + R_004^2*x*z^4
            + (-R_040)*x*y^4*z + (-2)*x^3*y*z^2 - x*y^3*z^2 + 2*R_004*x^2*y*z^3
            + x^3*y^2*z^2 - x^2*y*z^4 + P_004*x^2*z^5 + 2*R_004*x*y*z^5 + (-2*P_004*R_004)*x*z^6 + (-R_004^2)*y*z^6 + P_004*R_004^2*z^7
            + P_040*x*y^4*z^3 + R_040*y^5*z^3 + x^4*z^4 + 3*x^2*y^2*z^4 + (-P_004*R_040 - P_040*R_004)*y^4*z^4 + (-3*R_004)*x^3*z^5 + (-2*P_004)*x^2*y*z^5 + (-4*R_004)*x*y^2*z^5 + 3*R_004^2*x^2*z^6 + 2*P_004*R_004*x*y*z^6 + R_004^2*y^2*z^6 + (-R_004^3)*x*z^7
            + ... )
y + z^2 * ( - x + Q_004*z
            + x*y + (-R_004)*y*z
            + x^3 - x*y^2
            - x^2*z^2 + (Q_004 + R_004)*x*

In [4]:
print('***** Characteristic direction [0,1,1]\n')
X1_z.print_ttti_exp([0,1,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X1_z.get_sat_first_hom([0,1,0]))

***** Characteristic direction [0,1,1]

*** Expansion around the point [0, 1, 0] up to order 10

x + z^2 * (   y + (P_004 + P_040)*z
            + y^2 + (-R_040 - R_004)*x*z + 4*P_040*y*z
            - x^2*y + (-4*R_040)*x*y*z + 6*P_040*y^2*z
            + (-6*R_040)*x*y^2*z + 4*P_040*y^3*z + (-R_040 - R_004)*y*z^3 + (-P_004*R_040 - P_040*R_040 - P_004*R_004 - P_040*R_004)*z^4
            + (-4*R_040)*x*y^3*z + P_040*y^4*z - x*y^2*z^2 + (-P_004 - P_040)*x*y*z^3 + (-5*R_040 - R_004)*y^2*z^3 + (R_040^2 + 2*R_040*R_004 + R_004^2)*x*z^4 + (-4*P_004*R_040 - 8*P_040*R_040 - 4*P_040*R_004)*y*z^4
            + (-R_040)*x*y^4*z - x*y^3*z^2 + (2*R_040 + 2*R_004)*x^2*y*z^3 + (-4*P_040)*x*y^2*z^3 + (-10*R_040)*y^3*z^3 + (8*R_040^2 + 8*R_040*R_004)*x*y*z^4 + (-6*P_004*R_040 - 28*P_040*R_040 - 6*P_040*R_004)*y^2*z^4
            + x^3*y^2*z^2 + 8*R_040*x^2*y^2*z^3 + (-6*P_040)*x*y^3*z^3 + (-10*R_040)*y^4*z^3 + (28*R_040^2 + 12*R_040*R_004)*x*y^2*z^4 + (-4*P_004*R_040 - 56*P_040*R_040 - 4*P_040*R_004)

In [5]:
print('***** Characteristic direction [1,1,1]\n')
X1_z.print_ttti_exp([1,1,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X1_z.get_sat_first_hom([1,1,0]))

***** Characteristic direction [1,1,1]

*** Expansion around the point [1, 1, 0] up to order 10

x + z^2 * (   (P_004 + P_040 - R_040 - R_004)*z
            + (-2)*x*y + y^2 + (-R_040 - R_004)*x*z + (4*P_040 - 4*R_040)*y*z
            - x^2*y + (-4*R_040)*x*y*z + (6*P_040 - 6*R_040)*y^2*z
            + (-6*R_040)*x*y^2*z + (4*P_040 - 4*R_040)*y^3*z + (-P_004 - P_040 + R_040 + R_004)*y*z^3 + (-P_004*R_040 - P_040*R_040 + R_040^2 - P_004*R_004 - P_040*R_004 + 2*R_040*R_004 + R_004^2)*z^4
            + (-4*R_040)*x*y^3*z + (P_040 - R_040)*y^4*z + 2*x*y^2*z^2 - y^3*z^2 + (-P_004 - P_040 + 4*R_040 + 4*R_004)*x*y*z^3 + (-4*P_040 + 3*R_040 - R_004)*y^2*z^3 + (R_040^2 + 2*R_040*R_004 + R_004^2)*x*z^4 + (-4*P_004*R_040 - 8*P_040*R_040 + 8*R_040^2 - 4*P_040*R_004 + 8*R_040*R_004)*y*z^4
            + (-R_040)*x*y^4*z + 3*x^2*y^2*z^2 - x*y^3*z^2 + (2*R_040 + 2*R_004)*x^2*y*z^3 + (-4*P_040 + 16*R_040)*x*y^2*z^3 + (-6*P_040 + 2*R_040)*y^3*z^3 + (8*R_040^2 + 8*R_040*R_004)*x*y*z^4 + (-6*P_004*R_040 -

In [6]:
print('***** Characteristic direction [-1,1,1]\n')
X1_z.print_ttti_exp([-1,1,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X1_z.get_sat_first_hom([-1,1,0]))

***** Characteristic direction [-1,1,1]

*** Expansion around the point [-1, 1, 0] up to order 10

x + z^2 * (   (P_004 + P_040 + R_040 + R_004)*z
            + 2*x*y + y^2 + (-R_040 - R_004)*x*z + (4*P_040 + 4*R_040)*y*z
            - x^2*y + (-4*R_040)*x*y*z + (6*P_040 + 6*R_040)*y^2*z
            + (-6*R_040)*x*y^2*z + (4*P_040 + 4*R_040)*y^3*z + (P_004 + P_040 + R_040 + R_004)*y*z^3 + (-P_004*R_040 - P_040*R_040 - R_040^2 - P_004*R_004 - P_040*R_004 - 2*R_040*R_004 - R_004^2)*z^4
            + (-4*R_040)*x*y^3*z + (P_040 + R_040)*y^4*z + 2*x*y^2*z^2 + y^3*z^2 + (-P_004 - P_040 - 4*R_040 - 4*R_004)*x*y*z^3 + (4*P_040 + 3*R_040 - R_004)*y^2*z^3 + (R_040^2 + 2*R_040*R_004 + R_004^2)*x*z^4 + (-4*P_004*R_040 - 8*P_040*R_040 - 8*R_040^2 - 4*P_040*R_004 - 8*R_040*R_004)*y*z^4
            + (-R_040)*x*y^4*z + (-3)*x^2*y^2*z^2 - x*y^3*z^2 + (2*R_040 + 2*R_004)*x^2*y*z^3 + (-4*P_040 - 16*R_040)*x*y^2*z^3 + (6*P_040 + 2*R_040)*y^3*z^3 + (8*R_040^2 + 8*R_040*R_004)*x*y*z^4 + (-6*P_004*R_040 - 

In [7]:
print('***** Characteristic direction [0,1,0]\n')
X1_y.print_ttti_exp([0,0,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X1_y.get_sat_first_hom([0,0,0]))

***** Characteristic direction [0,1,0]

*** Expansion around the point [0, 0, 0] up to order 10

x + y^2 * (   P_040*y + z
            - z^2
            - x^4 + x^2*z^2
            + P_004*y*z^4
            + (-P_040)*x^3*y^3 - x^3*y^2*z + P_040*x*y^3*z^2 + x*y^2*z^3 + (-Q_004)*x*y*z^4
            + x^3*y^2*z^2 - x*y^2*z^4
            + (-P_040*Q_004)*y^4*z^4 + (-Q_004)*y^3*z^5
            + ... )
y + y^3 * (   x^3 - x*z^2
            + Q_004*y*z^4
            + ... )
z + y^2 * (   R_040*y
            + x*z
            - x*z^2
            - x^3*z + x*z^3
            + R_004*y*z^4
            + (-R_040)*x^3*y^3 + R_040*x*y^3*z^2 + (-Q_004)*y*z^5
            - x^4*y^2*z + x^2*y^2*z^3
            + x^4*y^2*z^2 - x^2*y^2*z^4 + (-Q_004*R_040)*y^4*z^4
            + ... )


*** Common factor of terms from degree [5, 6, 5] by component
*** Excluding the common factors [y^2, y^3, y^2]

In the x component: 1
In the y component: 1
In the z component: 1


*** Saturated homogeneous part of lower de

### Second blowup (of p5)

In [8]:
X2_x=X1_y.blowup([0,0,0],1,out_map=True)
print('***** Characteristic direction [1,0,0]\n')
X2_x.print_ttti_exp([0,0,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X2_x.get_sat_first_hom([0,0,0]))

***** Characteristic direction [1,0,0]

*** Expansion around the point [0, 0, 0] up to order 10

x + x^3*y^2 * (   P_040*y + z
                - x^3 - x*z^2
                + x^3*z^2
                + ... )
y + x^2*y^3 * (   (-P_040)*y - z
                + 2*x^3 + x*z^2
                + (-2)*x^3*z^2
                + ... )
z + x^2*y^2 * (   R_040*y
                + x*z + (-P_040)*y*z - z^2
                - x^2*z^2 + x*z^3
                + (-P_040*R_040)*x^2*y^4 + (-R_040)*x^2*y^3*z
                + ... )


*** Common factor of terms from degree [8, 8, 7] by component
*** Excluding the common factors [x^3*y^2, x^2*y^3, x^2*y^2]

In the x component: x
In the y component: x
In the z component: x


*** Saturated homogeneous part of lower degree

[0, 0, R_040*y]


### Third blowup (of p5,1)

In [9]:
X3_x=X2_x.blowup([0,0,0],1,out_map=True)
print('***** Characteristic direction [1,0,z0]\n')
X3_x.print_ttti_exp([0,0,z0],12,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X3_x.get_sat_first_hom([0,0,z0]))

***** Characteristic direction [1,0,z0]

*** Expansion around the point [0, 0, z0] up to order 12

x + x^6*y^2 * (   z0
                + P_040*y + z
                + (-z0^2 - 1)*x^2
                + (-2*z0)*x^2*z
                + z0^2*x^4 - x^2*z^2
                + ... )
y + x^5*y^3 * ( - 2*z0
                + (-2*P_040)*y + (-2)*z
                + (2*z0^2 + 3)*x^2
                + 4*z0*x^2*z
                + (-3*z0^2)*x^4 + 2*x^2*z^2
                + ... )
z + x^4*y^2 * (   (-2*z0^2 + z0)*x + R_040*y
                + (-2*P_040*z0)*x*y + (-4*z0 + 1)*x*z
                + (2*z0^3 - z0^2 + z0)*x^3 + (-2*P_040)*x*y*z + (-2)*x*z^2
                + (6*z0^2 - 2*z0 + 1)*x^3*z
                + (-z0^3)*x^5 + (6*z0 - 1)*x^3*z^2
                + (-3*z0^2)*x^5*z + 2*x^3*z^3
                + ... )


*** Common factor of terms from degree [11, 11, 9] by component
*** Excluding the common factors [x^6*y^2, x^5*y^3, x^4*y^2]

In the x component: x^2
In the y component: x^2
In the z comp

In [10]:
X3_z=X2_x.blowup([0,0,0],3,out_map=True)
print('***** Characteristic direction [0,0,1]\n')
X3_z.print_ttti_exp([0,0,0],15,sep_fact=True,rem_fact=True)
print('\n\n*** Saturated homogeneous part of lower degree\n')
print(X3_z.get_sat_first_hom([0,0,0]))

***** Characteristic direction [0,0,1]

*** Expansion around the point [0, 0, 0] up to order 15

x + x^3*y^2*z^4 * (   (-R_040)*y + 2*z
                    - x*z + 2*P_040*y*z
                    + (-2)*x*z^3
                    + x^2*z^3
                    - x^3*z^3
                    + ... )
y + x^2*y^3*z^4 * (   (-R_040)*y
                    - x*z
                    + x^2*z^3
                    + 2*x^3*z^3
                    + ... )
z + x^2*y^2*z^5 * (   R_040*y - z
                    + x*z + (-P_040)*y*z
                    + x*z^3
                    - x^2*z^3
                    + ... )


*** Common factor of terms from degree [12, 12, 12] by component
*** Excluding the common factors [x^3*y^2*z^4, x^2*y^3*z^4, x^2*y^2*z^5]

In the x component: x*z^3
In the y component: x^2*z^3
In the z component: x*z^3


*** Saturated homogeneous part of lower degree

[(-R_040)*x*y + 2*x*z, (-R_040)*y^2, R_040*y*z - z^2]


### Fourth blowup (of the line L)

In [11]:
X4_xy=X3_x.blowup_ax([0,0,0],[3],1,out_map=True)
print('***** Characteristic direction [1,0,0]\n')
X4_xy.print_ttti_exp([0,0,0],15,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous parts of lower degrees, saturated wrt x=0\n')
print([X4_xy.PolRing.ideal(x^2).reduce(ss) for ss in X4_xy.get_sat_first_hom([0,0,0], div=x, prec=3)]," + terms in x^2")

***** Characteristic direction [1,0,0]

*** Expansion around the point [0, 0, 0] up to order 15

x + x^8*y^2 * (   z
                - x^2 + P_040*x*y
                - x^2*z^2
                + ... )
y + x^7*y^3 * (   (-3)*z
                + 4*x^2 + (-3*P_040)*x*y
                + 3*x^2*z^2
                + ... )
z + x^7*y^2 * (   R_040*y + z
                + (-2)*z^2
                + x^2*z + (-2*P_040)*x*y*z
                - x^2*z^2
                + 2*x^2*z^3
                + ... )


*** Common factor of terms from degree [13, 13, 12] by component
*** Excluding the common factors [x^8*y^2, x^7*y^3, x^7*y^2]

In the x component: x^2*z^2
In the y component: x^2*z^2
In the z component: x*z


*** Homogeneous parts of lower degrees, saturated wrt x=0

[x*y^2*z, (-3*P_040)*x*y^4 + (-3)*y^3*z, (-2*P_040)*x*y^3*z + (-2)*y^2*z^2 + R_040*y^3 + y^2*z]  + terms in x^2


In [12]:
print('\n\n*** Homogeneous parts of lower degrees, saturated wrt xy=0, at a point [0,0,z0]\n')
print([X4_xy.PolRing.ideal([x^2,x*y]).reduce(ss).subs(z=z+z0).expand() for ss in X4_xy.get_sat_first_hom([0,0,0],div=x*y, prec=3)]," + terms in x^2,xy")



*** Homogeneous parts of lower degrees, saturated wrt xy=0, at a point [0,0,z0]

[x*z + x*z0, -3*y*z - 3*y*z0, R_040*y - 2*z^2 - 4*z*z0 - 2*z0^2 + z + z0]  + terms in x^2,xy


In [13]:
print('***** Characteristic direction [1,0,1/2]\n')
X4_xy.print_ttti_exp([0,0,1/2],15,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous part of lower degree, saturated\n')
print(X4_xy.get_sat_first_hom([0,0,1/2]))

***** Characteristic direction [1,0,1/2]

*** Expansion around the point [0, 0, 1/2] up to order 15

x + x^8*y^2 * (   1/2
                + z
                + (-5/4)*x^2 + P_040*x*y
                - x^2*z
                + 1/4*x^4 - x^2*z^2
                + x^4*z
                + ... )
y + x^7*y^3 * ( - 3/2
                + (-3)*z
                + 19/4*x^2 + (-3*P_040)*x*y
                + 3*x^2*z
                - x^4 + 3*x^2*z^2
                + (-4)*x^4*z
                + ... )
z + x^7*y^2 * (   R_040*y - z
                + 1/2*x^2 + (-P_040)*x*y + (-2)*z^2
                + 3/2*x^2*z + (-2*P_040)*x*y*z
                + (-1/8)*x^4 + 2*x^2*z^2
                + (-3/4)*x^4*z + 2*x^2*z^3
                + (-3/2)*x^4*z^2
                + ... )


*** Common factor of terms from degree [13, 13, 12] by component
*** Excluding the common factors [x^8*y^2, x^7*y^3, x^7*y^2]

In the x component: x^2
In the y component: x^2
In the z component: x


*** Homogeneous part of lower deg

In [14]:
X4_xx=X3_x.blowup_ax([0,0,0],[3],2,out_map=True)
print('***** Characteristic direction [0,1,0]\n')
X4_xx.print_ttti_exp([0,0,0],18,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous parts from degree %d to %d, saturated wrt y=0\n'%(list(X4_xx.get_first_hom())[0],list(X4_xx.get_first_hom())[0]+3))
print([ss.subs(y=0) for ss in X4_xx.get_sat_first_hom([0,0,0], div=y,prec=4)], " + terms in y")

***** Characteristic direction [0,1,0]

*** Expansion around the point [0, 0, 0] up to order 18

x + x^6*y^7 * (   3*P_040*y + 3*z
                + (-4)*x^2*y^2
                + ... )
y + x^5*y^8 * (   (-2*P_040)*y + (-2)*z
                + 3*x^2*y^2
                + ... )
z + x^4*y^7 * (   R_040
                + x*z
                + (-2*P_040)*x*y*z + (-2)*x*z^2
                + x^3*y^2*z
                - x^3*y^2*z^2
                + ... )


*** Common factor of terms from degree [16, 16, 14] by component
*** Excluding the common factors [x^6*y^7, x^5*y^8, x^4*y^7]

In the x component: x^2*y^2
In the y component: x^2*y^2
In the z component: x*z


*** Homogeneous parts from degree 11 to 14, saturated wrt y=0

[0, 0, x^5*z + R_040*x^4]  + terms in y


In [15]:
X4_zy=X3_z.blowup_ax([0,0,0],[1],3,out_map=True)
print('***** Characteristic direction [0,0,1]\n')
X4_zy.print_ttti_exp([0,0,0],18,sep_fact=True,rem_fact=True)
print('\n\n*** Constant part, saturated wrt xz=0, at a point [0,y0,0]\n')
print([(lambda x: factor(x) if x!=0 else 0)(X4_zy.PolRing.ideal([x,y,z]).reduce(ss)) for ss in X4_zy.get_sat_first_hom([0,y0,0],div=x*z, prec=4)])

***** Characteristic direction [0,0,1]

*** Expansion around the point [0, 0, 0] up to order 18

x + x^3*y^2*z^7 * (   2
                    - x + (-R_040)*y
                    + 2*P_040*y*z
                    + (-2)*x*z^2
                    + x^2*z^2
                    - x^3*z^2
                    + ... )
y + x^2*y^3*z^7 * (   1
                    + (-2)*x + (-2*R_040)*y
                    + P_040*y*z
                    - x*z^2
                    + 2*x^2*z^2
                    + 2*x^3*z^2
                    + ... )
z + x^2*y^2*z^8 * ( - 1
                    + x + R_040*y
                    + (-P_040)*y*z
                    + x*z^2
                    - x^2*z^2
                    + ... )


*** Common factor of terms from degree [15, 15, 15] by component
*** Excluding the common factors [x^3*y^2*z^7, x^2*y^3*z^7, x^2*y^2*z^8]

In the x component: x*z^2
In the y component: x*z^2
In the z component: x*z^2


*** Constant part, saturated wrt xz=0, at a point [0,y0,0]

[0, -2*

In [16]:
print('\n\n*** Homogeneous parts of lower degree, saturated wrt xz=0, at a point [0,1/2R_040,0]\n')
print([(ss*4*R_040^2).expand() for ss in X4_zy.get_sat_first_hom([0,1/(2*R_040),0],div=x*z, prec=1)])



*** Homogeneous parts of lower degree, saturated wrt xz=0, at a point [0,1/2R_040,0]

[3/2*x, -y - x/R_040 + 1/4*P_040*z/R_040^2, -1/2*z]


In [17]:
X4_zz=X3_z.blowup_ax([0,0,0],[1],2,out_map=True)
print('***** Characteristic direction [0,1,0]\n')
X4_zz.print_ttti_exp([0,0,0],18,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous part of lowest degree, saturated\n')
print(X4_zz.get_sat_first_hom([0,0,0],div=0, prec=1))

***** Characteristic direction [0,1,0]

*** Expansion around the point [0, 0, 0] up to order 18

x + x^3*y^7*z^4 * ( - R_040
                    + 2*z
                    - x*z + 2*P_040*y*z
                    + ... )
y + x^2*y^8*z^4 * ( - R_040
                    - x*z
                    + ... )
z + x^2*y^7*z^5 * (   2*R_040
                    - z
                    + 2*x*z + (-P_040)*y*z
                    + ... )


*** Common factor of terms from degree [17, 17, 17] by component
*** Excluding the common factors [x^3*y^7*z^4, x^2*y^8*z^4, x^2*y^7*z^5]

In the x component: 0
In the y component: 0
In the z component: 0


*** Homogeneous part of lowest degree, saturated

[(-R_040)*x, (-R_040)*y, 2*R_040*z]


### Further blow-up in p3

In [18]:
X1_p3=X1_z.translate([1,1,0],out_map=True)
lin_part_p3=X1_p3.get_sat_first_hom([0,0,0])
print('*** Homogeneous part of lowest degree, staurated')
print(lin_part_p3)

*** Homogeneous part of lowest degree, staurated
[(P_004 + P_040 - R_040 - R_004)*z, 2*x - y + (Q_004 - R_040 - R_004)*z, 0]


In case $\alpha=P^{(4)}(1,1,1)-R^{(4)}(1,1,1)\neq 0$, the set of singular directions has dimension 0.

In [19]:
M=Matrix([[l.monomial_coefficient(mon) for mon in [X1_p3.PolRing(x),X1_p3.PolRing(y),X1_p3.PolRing(z)]] for l in lin_part_p3])
print('*** Characteristid directions when alpha!=0')
for h in M.eigenvectors_right():
    print(list(h[1][0]))

*** Characteristid directions when alpha!=0
[0, 1, 0]
[1, 2, 0]


In [20]:
print('***** Blow-up at p3,1=[0,1,0]')
X5_p31=X1_p3.blowup([0,0,0],2,out_map=True)
print('***** Characteristic direction [0,1,0]\n')
X5_p31.print_ttti_exp([0,0,0],10,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous part of lowest degree, saturated\n')
print(X5_p31.get_sat_first_hom([0,0,0],div=0, prec=1))

***** Blow-up at p3,1=[0,1,0]
***** Characteristic direction [0,1,0]

*** Expansion around the point [0, 0, 0] up to order 10

x + y^2*z^2 * (   x + y + (P_004 + P_040 - R_040 - R_004)*z
                + (-2)*x^2 - x*y + (-Q_004 + R_040 + R_004)*x*z + (4*P_040 - 4*R_040)*y*z
                + x^2*y + 4*R_040*x*y*z + (6*P_040 - 6*R_040)*y^2*z
                + (-3)*x^3*y + 6*R_040*x*y^2*z + (4*P_040 - 4*R_040)*y^3*z
                + 4*R_040*x*y^3*z + (P_040 - R_040)*y^4*z + x*y^2*z^2 + y^3*z^2 + (P_004 + P_040 - R_040 - R_004)*y^2*z^3
                - x^4*y^2 + R_040*x*y^4*z + (-4)*x^2*y^2*z^2 + (-3)*x*y^3*z^2 + (-2*P_004 - 2*P_040 - 2*Q_004 + 4*R_040 + 4*R_004)*x*y^2*z^3 + (4*P_040 - Q_004 - 3*R_040 + R_004)*y^3*z^3 + (-P_004*Q_004 - P_040*Q_004 + P_004*R_040 + P_040*R_040 + Q_004*R_040 - R_040^2 + P_004*R_004 + P_040*R_004 + Q_004*R_004 - 2*R_040*R_004 - R_004^2)*y^2*z^4
                + ... )
y + y^3*z^2 * ( - 1
                + 2*x - y + (Q_004 - R_040 - R_004)*z
              

In [21]:
print('***** Characteristic direction [1,2,0]\n')
X5_p32=X5_p31.translate([1/2,0,0],out_map=True)
X5_p32.print_ttti_exp([0,0,0],8,sep_fact=True,rem_fact=True)
print('\n\n*** Homogeneous part of lowest degree, saturated\n')
print(X5_p32.get_sat_first_hom([0,0,0],div=0, prec=1))

***** Characteristic direction [1,2,0]

*** Expansion around the point [0, 0, 0] up to order 8

x + y^2*z^2 * ( - x + 3/8*y + (P_004 + P_040 - 1/2*Q_004 - 1/2*R_040 - 1/2*R_004)*z
                + (-2)*x^2 + (-9/4)*x*y + (-1/16)*y^2 + (-Q_004 + R_040 + R_004)*x*z + (4*P_040 - 2*R_040)*y*z
                + (-7/2)*x^2*y + (-1/2)*x*y^2 + 4*R_040*x*y*z + (6*P_040 - 3*R_040)*y^2*z
                + (-3)*x^3*y + (-3/2)*x^2*y^2 + 6*R_040*x*y^2*z + (4*P_040 - 2*R_040)*y^3*z
                + ... )
y + y^3*z^2 * (   2*x + (-3/4)*y + (Q_004 - R_040 - R_004)*z
                + 2*x*y + (-3/8)*y^2 + (-5*R_040 - R_004)*y*z
                + 3*x^2*y + (-1/4)*x*y^2 + (-10*R_040)*y^2*z
                + ... )
z + y^2*z^3 * (   (-2)*x + 7/4*y + (-Q_004 + R_040 + R_004)*z
                + (-2)*x*y + 7/8*y^2 + (6*R_040 + 2*R_004)*y*z
                + (-3)*x^2*y + 5/4*x*y^2 + 14*R_040*y^2*z
                + ... )


*** Common factor of terms from degree [7, 8, 8] by component
*** Excluding the common

### Change of coordinates at p1

In [22]:
M=Matrix([[1,1,0],[1,-1,0],[0,0,1]]).inverse()
X1_z_1=X1_z.lin_change([0,0,0],M,out_map=True)
X1_z_1.print_ttti_exp([0,0,0],8,sep_fact=True,rem_fact=True)

*** Expansion around the point [0, 0, 0] up to order 8

x + z^2 * ( - x + (P_004 + Q_004)*z
            + 3/4*x^2 + 1/4*y^2 + (-R_004)*x*z
            + (-1/8)*x^3 + 3/8*x^2*y + 5/8*x*y^2 + 1/8*y^3
            + (-1/2)*x^2*z^2 + (-1/2)*x*y*z^2 + (1/2*P_004 + 1/2*Q_004 + R_004)*x*z^3 + (1/2*P_004 + 1/2*Q_004)*y*z^3 + (-P_004*R_004 - Q_004*R_004)*z^4
            + 1/16*P_040*x^4*z + (-1/4*P_040)*x^3*y*z + 3/8*P_040*x^2*y^2*z + (-1/4*P_040)*x*y^3*z + 1/16*P_040*y^4*z + 5/8*x^3*z^2 + 3/8*x^2*y*z^2 + (-1/8)*x*y^2*z^2 + 1/8*y^3*z^2 + (-1/4*P_004 - 1/4*Q_004 - 5/4*R_004)*x^2*z^3 + (-1/2*R_004)*x*y*z^3 + (1/4*P_004 + 1/4*Q_004 - 1/4*R_004)*y^2*z^3 + R_004^2*x*z^4
            + (-1/16*R_040)*x^5*z + 1/4*R_040*x^4*y*z + (-3/8*R_040)*x^3*y^2*z + 1/4*R_040*x^2*y^3*z + (-1/16*R_040)*x*y^4*z + (-1/4)*x^4*z^2 + 1/8*x^3*y*z^2 + 5/8*x^2*y^2*z^2 + 3/8*x*y^3*z^2 + 1/8*y^4*z^2 + 3/8*R_004*x^3*z^3 + (-3/8*R_004)*x^2*y*z^3 + (-7/8*R_004)*x*y^2*z^3 + (-1/8*R_004)*y^3*z^3
            + ... )
y + z^2 * (   y +

In [23]:
M=Matrix([[1,0,-(P_004 + Q_004)],[0,1,(P_004 - Q_004)],[0,0,1]]).inverse()
X1_z_2=X1_z_1.lin_change([0,0,0],M,out_map=True)
X1_z_2.print_ttti_exp([0,0,0],9,sep_fact=True,rem_fact=True)

*** Expansion around the point [0, 0, 0] up to order 9

x + z^2 * ( - x
            + 3/4*x^2 + 1/4*y^2 + (2*P_004 + 2*Q_004 - R_004)*x*z + Q_004*y*z + (P_004^2 + 2*P_004*Q_004 + 2*Q_004^2 - 2*P_004*R_004 - 2*Q_004*R_004)*z^2
            + (-1/8)*x^3 + 3/8*x^2*y + 5/8*x*y^2 + 1/8*y^3 + (-P_004 - 1/4*Q_004)*x^2*z + (-1/2*P_004 + 2*Q_004)*x*y*z + (1/2*P_004 + 5/4*Q_004)*y^2*z + (-P_004^2 - 3*P_004*Q_004 + 1/2*Q_004^2)*x*z^2 + (-P_004^2 + 5/2*Q_004^2)*y*z^2 + (-2*P_004^2*Q_004 - 2*P_004*Q_004^2 + Q_004^3)*z^3
            + (-1/2)*x^2*z^2 + (-1/2)*x*y*z^2 + (-Q_004 + R_004)*x*z^3
            + 1/16*P_040*x^4*z + (-1/4*P_040)*x^3*y*z + 3/8*P_040*x^2*y^2*z + (-1/4*P_040)*x*y^3*z + 1/16*P_040*y^4*z + (1/2*P_004*P_040 + 5/8)*x^3*z^2 + (-3/2*P_004*P_040 + 3/8)*x^2*y*z^2 + (3/2*P_004*P_040 - 1/8)*x*y^2*z^2 + (-1/2*P_004*P_040 + 1/8)*y^3*z^2 + (3/2*P_004^2*P_040 + 5/4*P_004 + 2*Q_004 - 5/4*R_004)*x^2*z^3 + (-3*P_004^2*P_040 + P_004 + 1/2*Q_004 - 1/2*R_004)*x*y*z^3 + (3/2*P_004^2*P_040 - 1/4*P_004

In the x component: 1
In the y component: 1
In the z component: x^4*z + (-4)*x^3*y*z + 6*x^2*y^2*z + (-4)*x*y^3*z + y^4*z + 8*P_004*x^3*z^2 + (-24*P_004)*x^2*y*z^2 + 24*P_004*x*y^2*z^2 + (-8*P_004)*y^3*z^2 + 24*P_004^2*x^2*z^3 + (-48*P_004^2)*x*y*z^3 + 24*P_004^2*y^2*z^3 + 32*P_004^3*x*z^4 + (-32*P_004^3)*y*z^4 + 16*P_004^4*z^5


### Change of coordinates at p3,2 and blow-up

In [24]:
M=Matrix([[-1,3/8,(P_004 + P_040 - 1/2*Q_004 - 1/2*R_040)],[0,1,0],[0,0,1]])
X5_p32_1=X5_p32.lin_change([0,0,0],M,out_map=True)
X5_p32_1.print_ttti_exp([0,0,0],9,sep_fact=True,rem_fact=True)


*** Expansion around the point [0, 0, 0] up to order 9

x + y^2*z^2 * ( - x + 1/2*R_004*z
                + 2*x^2 + (-9/2)*x*y + 19/16*y^2 + (-2*P_004 - 2*P_040 + 2*R_040 + R_004)*x*z + (11/2*P_004 + 3/2*P_040 - 2*Q_004 - 3/2*R_040 - 3/4*R_004)*y*z
                + 7/2*x^2*y + (-31/8)*x*y^2 + 105/128*y^3 + (-5*P_004 - 5*P_040 + 5/2*Q_004 + 13/2*R_040)*x*y*z + (4*P_004 - 2*P_040 - 2*Q_004 - 19/8*R_040 - 3/8*R_004)*y^2*z + (3/2*P_004^2 + 3*P_004*P_040 + 3/2*P_040^2 - 3/2*P_004*Q_004 - 3/2*P_040*Q_004 + 3/8*Q_004^2 + 1/2*P_004*R_040 + 1/2*P_040*R_040 - 1/4*Q_004*R_040 - 5/8*R_040^2 + 2*P_004*R_004 + 2*P_040*R_004 - Q_004*R_004 - R_040*R_004)*y*z^2
                + (-3)*x^3*y + 6*x^2*y^2 + (-201/64)*x*y^3 + 63/128*y^4 + (6*P_004 + 6*P_040 - 3*Q_004 - 3*R_040)*x^2*y*z + (-11*P_004 - 11*P_040 + 11/2*Q_004 + 23/2*R_040)*x*y^2*z + (51/16*P_004 - 13/16*P_040 - 51/32*Q_004 - 179/32*R_040)*y^3*z + (-3*P_004^2 - 6*P_004*P_040 - 3*P_040^2 + 3*P_004*Q_004 + 3*P_040*Q_004 - 3/4*Q_004^2 + 3*P_004*R_

In [25]:
print('***** Blow-up at p3,2 - z-chart')
X5_p32_1_z=X5_p32_1.blowup([0,0,0],3,out_map=True)
X5_p32_1_z.print_ttti_exp([0,0,0],12,rem_fact=True,sep_fact=True)

***** Blow-up at p3,2 - z-chart
*** Expansion around the point [0, 0, 0] up to order 12

x + y^2*z^4 * (   1/2*R_004
                - x
                + (11/2*P_004 + 3/2*P_040 - 2*Q_004 - 3/2*R_040 - 3/4*R_004)*y*z
                + (-11/2)*x*y*z + 19/16*y^2*z + (3/2*P_004^2 + 3*P_004*P_040 + 3/2*P_040^2 - 3/2*P_004*Q_004 - 3/2*P_040*Q_004 + 3/8*Q_004^2 + 1/2*P_004*R_040 + 1/2*P_040*R_040 - 1/4*Q_004*R_040 - 5/8*R_040^2 + 2*P_004*R_004 + 2*P_040*R_004 - Q_004*R_004 - R_040*R_004)*y*z^2
                + (-3*P_004 - 3*P_040 + 3/2*Q_004 - 1/2*R_040 - 2*R_004)*x*y*z^2 + (4*P_004 - 2*P_040 - 2*Q_004 - 19/8*R_040 - 3/8*R_004)*y^2*z^2
                + 3/2*x^2*y*z^2 + (-4)*x*y^2*z^2 + 105/128*y^3*z^2 + (5*P_004^2 + 10*P_004*P_040 + 5*P_040^2 - 5*P_004*Q_004 - 5*P_040*Q_004 + 5/4*Q_004^2 + 3*P_004*R_040 + 3*P_040*R_040 - 3/2*Q_004*R_040 - 11/4*R_040^2)*y^2*z^3
                + (-10*P_004 - 10*P_040 + 5*Q_004 - 3*R_040)*x*y^2*z^3 + (51/16*P_004 - 13/16*P_040 - 51/32*Q_004 - 179/32*R_040)*y

### Change of coordinates at q1

In [26]:
M=Matrix([[0,R_040,1],[1,0,0],[0,1,0]]).inverse()
X4_xy_1=X4_xy.lin_change([0,0,0],M,out_map=True)
X4_xy_1.print_ttti_exp([0,0,0],16,rem_fact=True,sep_fact=True)

*** Expansion around the point [0, 0, 0] up to order 16

x + y^7*z^2 * (   x
                + (-2)*x^2 + R_040*x*z + R_040^2*z^2
                + x*y^2 + (-2*P_040)*x*y*z + 3*R_040*y^2*z + (-P_040*R_040)*y*z^2
                - x^2*y^2 + 2*R_040*x*y^2*z + (-R_040^2)*y^2*z^2
                + 2*x^3*y^2 + (-3*R_040)*x^2*y^2*z + R_040^3*y^2*z^3
                - x^3*y^4 + (-R_040)*x^2*y^4*z + 5*R_040^2*x*y^4*z^2 + (-3*R_040^3)*y^4*z^3
                + ... )
y + y^8*z^2 * (   x + (-R_040)*z
                - y^2 + P_040*y*z
                - x^2*y^2 + 2*R_040*x*y^2*z + (-R_040^2)*y^2*z^2
                + x^2*y^4 + (-2*R_040)*x*y^4*z + R_040^2*y^4*z^2
                + ... )
z + y^7*z^3 * (   (-3)*x + 3*R_040*z
                + 4*y^2 + (-3*P_040)*y*z
                + 3*x^2*y^2 + (-6*R_040)*x*y^2*z + 3*R_040^2*y^2*z^2
                + (-4)*x^2*y^4 + 8*R_040*x*y^4*z + (-4*R_040^2)*y^4*z^2
                + ... )


*** Common factor of terms from degree [12, 13, 13] by component
*** Exc